# Assignment 1.2: Word2vec preprocessing (20 points)

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make **Batcher** class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size, 2*window_size)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']
```

In [0]:
import numpy as np
from torch.utils.data import DataLoader, Dataset
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
with open("/content/gdrive/My Drive/DeepPavlov/text8/text8","r") as f:
  data = f.read()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from string import punctuation
corpus = ''.join([c.lower() for c in data if c not in punctuation+'«»']).split()

In [0]:
# считаем слова и сортируем по убыванию количества вхождений
def counter(words):
  count_words = Counter(words)
  total_words = len(words)
  sorted_words = count_words.most_common(total_words)
  print('total words: {}'.format(total_words))
  print(sorted_words[150:200])
  less_10, more_10 = 0, 0
  for word,freq in sorted_words:
    if freq>10:
        more_10+=1
    if freq<10:
        less_10+=1
  print('more then 10: {}, less then 10: {}'.format(more_10,less_10))
  return sorted_words, more_10

In [0]:
# назначаю каждому токену номер
def make_dicts(sorted_words, dimensionality):
  word2index = {w:i+1 for i, (w,c) in enumerate(sorted_words[:(dimensionality-1)])}   
  word2index['UNK'] = 0
  index2word = {i+1:w for i, (w,c) in enumerate(sorted_words[:(dimensionality-1)])}
  index2word[0] = 'UNK'
  return word2index, index2word

In [0]:
class Data_Processing_CBOW():
  def __init__(self, corpus, window_size):
    self.corpus = corpus
    self.window_size = window_size
    self.dimensionality = np.nan


  def get_data(self):
    sorted_words, self.dimensionality = counter(self.corpus)
    word2index, index2word = make_dicts(sorted_words, self.dimensionality)
    # собственно делаем X,y
    X,y = [],[]
    num_sent = [word2index[x] if x in word2index.keys() else 0 for x in self.corpus]
    for cur_idx,num in enumerate(num_sent[self.window_size:-self.window_size]):
      idx = cur_idx+self.window_size
      y.append(num)
      X.append(num_sent[idx-self.window_size:idx]+num_sent[idx+1:idx+self.window_size+1])

    return np.array(X), np.array(y), word2index, index2word

In [14]:
X, y, word2index, index2word = Data_Processing_CBOW(corpus, 5).get_data()

total words: 17005207
[('french', 8736), ('before', 8700), ('general', 8659), ('what', 8581), ('t', 8491), ('against', 8432), ('n', 8372), ('high', 8337), ('links', 8312), ('could', 8304), ('based', 8244), ('those', 8209), ('now', 8206), ('second', 8110), ('de', 8002), ('music', 7987), ('another', 7933), ('large', 7898), ('she', 7896), ('f', 7878), ('external', 7862), ('german', 7858), ('different', 7797), ('modern', 7790), ('great', 7770), ('do', 7763), ('common', 7698), ('set', 7682), ('list', 7672), ('south', 7628), ('series', 7611), ('major', 7585), ('game', 7553), ('power', 7522), ('long', 7488), ('country', 7481), ('king', 7456), ('law', 7435), ('group', 7417), ('film', 7400), ('still', 7378), ('until', 7368), ('north', 7328), ('international', 7262), ('term', 7219), ('we', 7118), ('end', 7113), ('book', 7110), ('found', 7043), ('own', 7034)]
more then 10: 44611, less then 10: 206720


In [0]:
class My_Dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx, :], self.y[idx]

In [0]:
batch_size=512
batcher_train = DataLoader(My_Dataset(X, y), batch_size=batch_size, shuffle=False)